In [11]:
import pandas as pd
import matplotlib.pyplot as pyplt
%pylab inline
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.linear_model import Ridge, LinearRegression, LassoLarsCV, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn import preprocessing
import seaborn as sns
sns.set(color_codes=True)
from scipy.stats.stats import pearsonr 

from azureml import Workspace

ws = Workspace()
ds = ws.datasets['county_facts_dictionary.csv']
frame = ds.to_dataframe()

trainDF = ws.datasets['ames_train.csv'].to_dataframe();
testDF = ws.datasets['ames_test.csv'].to_dataframe();
sampleSubmissionDF = ws.datasets['ames_sample_submission.csv'].to_dataframe();

all_data = pd.concat((trainDF.loc[:,'MSSubClass':'SaleCondition'],
                      testDF.loc[:,'MSSubClass':'SaleCondition']))
y = trainDF.SalePrice
trainDF.SalePrice = log(y)

Populating the interactive namespace from numpy and matplotlib


In [12]:
all_data.loc[all_data.PoolQC.isnull(), 'PoolQC']='NA'


# In[212]:

all_data = all_data.replace({'PoolQC': {'Ex': 4,
                                            'Gd': 3,
                                            'TA': 2,
                                            'Fa': 1,
                                           'NA':0
                                            }
        })


# ### Alley

# In[213]:

all_data.loc[all_data.Alley.isnull(), 'Alley']='NA'


# ### Fence

# In[214]:

all_data.loc[all_data.Fence.isnull(), 'Fence']='NA'


# In[215]:

all_data = all_data.replace({'Fence': {'GdPrv': 4,
                                            'MnPrv': 3,
                                            'GdWo': 2,
                                            'MnWw': 1,
                                           'NA':0
                                            }
        })


# ### FirePlaceQu

# In[216]:

all_data.loc[all_data.FireplaceQu.isnull(), 'FireplaceQu']='NA'


# In[217]:

all_data = all_data.replace({'FireplaceQu': {'Ex': 5,
                                            'Gd': 4,
                                            'TA': 3,
                                            'Fa': 2,
                                            'Po': 1,
                                           'NA':0
                                            }
        })


# ### Utilities

# In[218]:

all_data.loc[all_data.Utilities.isnull(), 'Utilities']='AllPub'


# ### Kitchen Qualitty

# In[221]:

all_data.loc[all_data.KitchenQual.isnull(), 'KitchenQual']='TA'


# In[222]:

all_data = all_data.replace({ 'KitchenQual': {'Ex': 5,
                                            'Gd': 4,
                                            'TA': 3,
                                            'Fa': 2,
                                            'Po': 1
                                            }
        })


# ### SaleType

# In[223]:

all_data.loc[all_data.SaleType.isnull(), 'SaleType']='WD'


# ### MasVnrArea

# In[224]:

all_data.loc[all_data.MasVnrArea.isnull(), 'MasVnrArea']=0


# ### MasVnrType

# In[225]:

all_data.loc[all_data.MasVnrType.isnull(), 'MasVnrType']='None'


# ### Basement

# In[226]:

for c in ['TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtHalfBath', 'BsmtFullBath']:
    all_data.loc[all_data[c].isnull(), c]=0


# In[227]:

indexes = all_data['BsmtFinType1'].isnull()
for c in ['BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'BsmtCond', 'BsmtExposure']:
    all_data.loc[indexes, c]='NA'


# In[228]:

indexes = all_data['BsmtFinType1'].isnull()
for c in ['BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'BsmtCond', 'BsmtExposure']:
    all_data.loc[indexes, c]='NA'
#We filtered on BsmtFinType1 because it had least no of Nulls. Now there would 2 more Nulls in BsmtCond, need to check it further

all_data.loc[all_data.BsmtCond.isnull(), 'BsmtCond']='TA'
all_data = all_data.replace({ 'BsmtCond': {'Ex': 5,
                                            'Gd': 4,
                                            'TA': 3,
                                            'Fa': 2,
                                            'Po': 1,
                                           'NA':0
                                            }
        })


# In[231]:


all_data.loc[all_data.BsmtQual.isnull(), 'BsmtQual']='TA'
all_data = all_data.replace({ 'BsmtQual': {'Ex': 5,
                                            'Gd': 4,
                                            'TA': 3,
                                            'Fa': 2,
                                            'Po': 1,
                                           'NA':0
                                            }
        })



# In[234]:

all_data.loc[all_data.BsmtExposure.isnull(), 'BsmtExposure']='No'
all_data = all_data.replace({ 'BsmtExposure': {'Gd': 4,
                                            'Av': 3,
                                            'Mn': 2,
                                            'No': 1,
                                              'NA':0
                                            }
        })


# In[235]:

all_data.loc[all_data.BsmtFinType2.isnull(), 'BsmtFinType2']='Unf'
all_data = all_data.replace({ 'BsmtFinType2': {'GLQ': 6,
                                            'ALQ': 5,
                                            'BLQ': 4,
                                            'Rec': 3,
                                            'LwQ': 2,
                                              'Unf':1,
                                              'NA':0
                                            }
        })


# ### MSZoning

# In[237]:

all_data.loc[all_data.MSZoning.isnull(), 'MSZoning']='RL'


# ### Functional

# In[238]:

all_data.loc[all_data.Functional.isnull(), 'Functional']='Typ'
all_data = all_data.replace({ 'Functional': {'Typ': 7,
                                            'Min1': 6,
                                            'Min2': 5,
                                            'Mod': 4,
                                            'Maj1': 3,
                                              'Maj2':2,
                                              'Sev':1,
                                             'Sal':0
                                            }
        })


# In[239]:

all_data.loc[all_data.Electrical.isnull(), 'Electrical']='SBrkr'


# ### LotFrontage

# In[240]:

nullIndex = all_data.LotFrontage.isnull()


# In[241]:

nonNullIndex = all_data.LotFrontage.notnull()


# In[242]:

X_Train = np.sqrt(all_data.LotArea[nonNullIndex])


# In[243]:

Y_Train = all_data.LotFrontage[nonNullIndex]


# In[244]:

y= np.array(Y_Train)


# In[245]:

b=np.array(X_Train)
x = b.reshape(len(b),1)


# In[246]:

model = LinearRegression()


# In[247]:

model.fit(x, y)


# In[248]:

X_Test = np.sqrt(all_data.LotArea[nullIndex])
b=np.array(X_Test)
xt = b.reshape(len(b),1)
yPred = model.predict(xt)


# In[249]:

all_data.loc[nullIndex, 'LotFrontage']=yPred


# In[250]:

all_data.LotFrontage.isnull().sum()


# ### Garage

# In[251]:

garage = all_data[['GarageCond', 'GarageQual', 'GarageYrBlt', 'GarageFinish', 'GarageType', 'GarageCars', 'GarageArea']]


# In[252]:

garage.head()


# In[253]:

garage.GarageYrBlt.isnull().sum()


# In[254]:

garage[garage.GarageYrBlt.isnull()].head()


# So when garage year blt is null, we can say that there is no garage.
# Looking at the data description it seems that GarageType, GarageFinish, GarageQuality, GarageCondition has NA values signifying No Garrage. Replacing them.

# In[255]:

all_data.loc[all_data.GarageYrBlt.isnull(), 'GarageCond']='NA'


# In[256]:

all_data.loc[all_data.GarageYrBlt.isnull(), 'GarageFinish']='NA'


# In[257]:

all_data.loc[all_data.GarageYrBlt.isnull(), 'GarageQual']='NA'


# In[258]:

all_data.loc[all_data.GarageYrBlt.isnull(), 'GarageType']='NA'


# Now how to impute garage year built, when garage is not built. I think we should check the relation between Garage year build and sale price

# In[259]:

# In[260]:

trainDF['HasGarage']=trainDF.GarageYrBlt.apply(lambda x:0 if math.isnan(x) else 1)



# In[262]:

all_data.loc[all_data.GarageYrBlt.isnull(), 'GarageYrBlt']=0
all_data['HasGarage']=all_data.GarageYrBlt.apply(lambda x:0 if math.isnan(x) else 1)


# We clearly see that house that garage has more price. Now how to impute this value.

# In[263]:

garage = all_data[['GarageCond', 'GarageQual', 'GarageYrBlt', 'GarageFinish', 'GarageType', 'GarageCars', 'GarageArea']]


# In[264]:

garage[garage.GarageCars.isnull()]


# We will make them zero. We did not do it earlier because there were some cases when garageYrBlt was Null but Garage Cars where not null. Probably they refered to detached garage type. Lets have a look

# In[265]:

ind = garage.GarageYrBlt==0


# In[266]:

(garage.GarageCars[ind]!=0).sum()


# In[267]:

(garage.GarageArea[ind]!=0).sum()


# Okay so these are just two cases, we will make them all zero.

# In[268]:

all_data.loc[all_data.GarageCars.isnull(), ('GarageCars', 'GarageArea')]=0

all_data = pd.get_dummies(all_data)